# Postprocess predictions and create a submission

In [19]:
#@title Submission notes
submission_message = "YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15 + binary classifier on 1024 res and 0.8 thresh " #@param {type:"string"}

In [20]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, INPUT_FOLDER, WORK_FOLDER
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/output', '/kaggle/input', '/kaggle/working']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [21]:
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER_ORIGINAL_PNG = INPUT_FOLDER / "vinbigdata-chest-xray-original-png"
INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_BINARY = WORK_FOLDER / "vbdbinary"

## Get data from Kaggle

In [22]:
# Version notes: YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15
!kaggle datasets download "witalia/vbdyolo-out-newest/version/5" -p {INPUT_FOLDER_YOLO_OUT} --unzip --force

100% 155M/155M [00:01<00:00, 126MB/s]



In [23]:
# Version notes: EffNet, 1024 resolution, threshold 0.8
!kaggle datasets download "witalia/vbdbinary/version/4" -p {INPUT_FOLDER_BINARY} --unzip --force

 94% 97.0M/104M [00:00<00:00, 148MB/s]
100% 104M/104M [00:00<00:00, 176MB/s] 


## Process YOLO output

In [24]:
def read_prediction_labels(filename: Path, image_w: int, image_h: int):
    if not filename.exists():
        return "14 1 0 0 1 1"

    labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
    labels.columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = labels["x_centre"] - labels["bw"] / 2
    labels["y_min"] = labels["y_centre"] - labels["bh"] / 2
    labels["x_max"] = labels["x_centre"] + labels["bw"] / 2
    labels["y_max"] = labels["y_centre"] + labels["bh"] / 2
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert(labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

In [25]:
results_yolo_df = pd.DataFrame(columns=["image_id", "PredictionString"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    prediction_str = read_prediction_labels(
        INPUT_FOLDER_YOLO_OUT / "labels_pred" / f"{image_id}.txt", image_dims["dim0"], image_dims["dim1"]
    )
    results_yolo_df = results_yolo_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_yolo_df.sample(10)

,image_id,PredictionString
2769,ec2176e084f94ee8f9fc662354262062,14 1 0 0 1 1
2393,cc9f70916dd600695bd5364312472249,0 0.389648 1268 639 1479 878 3 0.420898 953 13...
1635,8dec5497ecc246766acfba5a4be4e619,0 0.505859 1032 652 1248 911
1284,71b4eda82984d225fdddc3a4d1d32ad5,14 1 0 0 1 1
1388,7919e0bc92270c1acbd8b124aa5377ef,11 0.220703 1311 310 1628 410 13 0.488037 615 ...
2793,eea860baa10cf65d1fabf38fad5c910e,3 0.37793 865 1330 1910 1761
2945,fc460aa7f3cdc18f224d1de7aa405893,0 0.321289 1271 787 1590 1161 3 0.457275 864 1...
867,4ef47d5bbd7105f7c7558bbb51735d36,11 0.156616 1305 275 1834 574 11 0.355225 492 ...
1944,a8eea443a1b6ff5a66a5b3d9ca3b43ec,14 1 0 0 1 1
405,26a5285339df294ce288663b092c23c3,14 1 0 0 1 1


## Merge with Binary classifier output

In [26]:
results_binary_df = pd.read_csv(INPUT_FOLDER_BINARY / "prediction.csv")
display(results_binary_df.head())

results_df = results_yolo_df.merge(results_binary_df, on="image_id")
results_df.loc[results_df["class_name"] == "normal", "PredictionString"] = "14 1 0 0 1 1"
results_df = results_df.drop(columns=["class_name"])

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.head())

,image_id,class_name
0,d0615f853a7deeec90f8b1bf30269fcc,normal
1,6637c08da9b3bce162e3aa689da14574,abnormal
2,b65467fc097115261fe11d2a99ccb5cd,abnormal
3,7742abe5bd817ba643d38fb517e35b24,normal
4,2790d36fa3cd4d3fa05229d1c693d499,abnormal


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.583008 1269 629 1538 919
2,008bdde2af2462e86fd373a445d0f4cd,3 0.438232 1142 1444 2028 1804 0 0.552246 1507...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,3 0.510254 724 1381 1750 1756


## Submit to Kaggle

In [27]:
!kaggle competitions submit \
    vinbigdata-chest-xray-abnormalities-detection \
    -f {WORK_FOLDER}/submission.csv \
    -m "{submission_message}"
!sleep 10
!kaggle competitions submissions vinbigdata-chest-xray-abnormalities-detection

100% 172k/172k [00:00<00:00, 817kB/s]
Successfully submitted to VinBigData Chest X-ray Abnormalities DetectionWarning: Looks like you're using an outdated API Version, please consider updating (server 1.5.12 / client 1.5.4)
fileName               date                 description                                                                                                                      status    publicScore  privateScore  
---------------------  -------------------  -------------------------------------------------------------------------------------------------------------------------------  --------  -----------  ------------  
submission.csv         2021-03-17 16:17:50  YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15 + binary classifier on 1024 res and 0.8 thresh   complete  0.107        None          
submission.csv         2021-03-17 16:03:45  YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15 + binary classifier